In [4]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import pickle

In [5]:
# LOAD data

In [6]:
teds = pd.read_csv('./data/ted_main.csv',header=0)
transcripts = pd.read_csv('./data/transcripts.csv',header=0)

In [29]:
teds.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [26]:
teds['description'].head(5)

0    Sir Ken Robinson makes an entertaining and pro...
1    With the same humor and humanity he exuded in ...
2    New York Times columnist David Pogue takes aim...
3    In an emotionally charged talk, MacArthur-winn...
4    You've never seen data presented like this. Wi...
Name: description, dtype: object

In [10]:
print(len(teds))

2550


In [11]:
teds_keys = [k for k in teds]
transcripts_keys = [k for k in transcripts]

In [12]:
print(teds_keys)
print(transcripts_keys)

['comments', 'description', 'duration', 'event', 'film_date', 'languages', 'main_speaker', 'name', 'num_speaker', 'published_date', 'ratings', 'related_talks', 'speaker_occupation', 'tags', 'title', 'url', 'views']
['transcript', 'url']


In [13]:
# ALLIGN data

In [14]:
tmp_url_list = []
tmp_x_list = []
for i in range(len(transcripts)):
    tmp_url_list.append(transcripts['url'][i])
    tmp_x_list.append(transcripts['transcript'][i])

In [15]:
data1 = []
non_align = 0
for i in range(len(teds)):
    dt = {}
    for k in teds_keys:
        if k in 'description event main_speaker name title url speaker_occupation'.split():
            dt[k] = teds[k][i]
        elif k in 'ratings related_talks tags'.split():
            dt[k] = eval(teds[k][i])
        else:
            dt[k] = int(teds[k][i])
        if k == 'url':
            try:
                j = tmp_url_list.index(teds[k][i])
                dt['transcripts'] = tmp_x_list[j]
            except:
                non_align = non_align + 1
    data1.append(dt)

In [16]:
print(len(data1))
data2 = [d for d in data1 if 'transcripts' in d]
print(len(data2))

2550
2464


In [17]:
# Build rating dictionaries
rating_dictionary = set()
for d in data2:
    ratings = d['ratings']
    for r in ratings:
        rating_dictionary.add(r['name'])
        
print(rating_dictionary)
print(len(rating_dictionary))
for d in data2:
    ratings = d['ratings']
    rate_dict = {}
    rate_sum = 0
    rating_rank = []
    for r in ratings:
        rate_dict[r['name']] = r['count']
        rate_sum = rate_sum + r['count']
    rate_percentage = {}
    for r in rating_dictionary:
        if r not in rate_dict:
            rate_dict[r] = 0
        rate_percentage[r] = rate_dict[r]/rate_sum
        rating_rank.append([rate_dict[r], r])
    d['rate_dict']=rate_dict
    d['rate_percentage']=rate_percentage
    rating_rank=sorted(rating_rank, reverse=True)
    rating_rank=[y for x,y in rating_rank]
    d['rate_rank']=rating_rank

set(['Beautiful', 'Funny', 'Inspiring', 'Obnoxious', 'OK', 'Fascinating', 'Informative', 'Persuasive', 'Longwinded', 'Ingenious', 'Jaw-dropping', 'Unconvincing', 'Courageous', 'Confusing'])
14


In [18]:
print(data2[0]['rate_dict'])
print(data2[0]['rate_rank'])

{'Beautiful': 4573, 'Funny': 19645, 'Inspiring': 24924, 'Obnoxious': 209, 'OK': 1174, 'Fascinating': 10581, 'Informative': 7346, 'Persuasive': 10704, 'Longwinded': 387, 'Ingenious': 6073, 'Jaw-dropping': 4439, 'Unconvincing': 300, 'Courageous': 3253, 'Confusing': 242}
['Inspiring', 'Funny', 'Persuasive', 'Fascinating', 'Informative', 'Ingenious', 'Beautiful', 'Jaw-dropping', 'Courageous', 'OK', 'Longwinded', 'Unconvincing', 'Confusing', 'Obnoxious']


In [19]:
with open('ted_data.pkl','wb+') as f:
    pickle.dump(data2, f)

In [65]:
import pandas as pd
data = pd.DataFrame(data2)

In [99]:
data.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,rate_percentage,rate_rank,ratings,related_talks,speaker_occupation,tags,title,transcripts,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,"{u'Beautiful': 0, u'Funny': 0, u'Inspiring': 0...","[Inspiring, Funny, Persuasive, Fascinating, In...","[{u'count': 19645, u'id': 7, u'name': u'Funny'...","[{u'viewed_count': 7266103, u'hero': u'https:/...",Author/educator,"[children, creativity, culture, dance, educati...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,"{u'Beautiful': 0, u'Funny': 0, u'Inspiring': 0...","[Funny, Informative, Inspiring, Persuasive, Un...","[{u'count': 544, u'id': 7, u'name': u'Funny'},...","[{u'viewed_count': 1751408, u'hero': u'https:/...",Climate advocate,"[alternative energy, cars, climate change, cul...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,...,"{u'Beautiful': 0, u'Funny': 0, u'Inspiring': 0...","[Funny, Informative, Persuasive, Inspiring, In...","[{u'count': 964, u'id': 7, u'name': u'Funny'},...","[{u'viewed_count': 4843421, u'hero': u'https:/...",Technology columnist,"[computers, entertainment, interface design, m...",Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,"{u'Beautiful': 0, u'Funny': 0, u'Inspiring': 0...","[Inspiring, Courageous, Persuasive, Informativ...","[{u'count': 760, u'id': 3, u'name': u'Courageo...","[{u'viewed_count': 702642, u'hero': u'https://...",Activist for environmental justice,"[MacArthur grant, activism, business, cities, ...",Greening the ghetto,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,"{u'Beautiful': 0, u'Funny': 0, u'Inspiring': 0...","[Informative, Fascinating, Jaw-dropping, Ingen...","[{u'count': 3202, u'id': 9, u'name': u'Ingenio...","[{u'viewed_count': 1345319, u'hero': u'https:/...",Global health expert; data visionary,"[Africa, Asia, Google, demo, economics, global...",The best stats you've ever seen,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [71]:
data['rate_dict'][0]

{'Beautiful': 4573,
 'Confusing': 242,
 'Courageous': 3253,
 'Fascinating': 10581,
 'Funny': 19645,
 'Informative': 7346,
 'Ingenious': 6073,
 'Inspiring': 24924,
 'Jaw-dropping': 4439,
 'Longwinded': 387,
 'OK': 1174,
 'Obnoxious': 209,
 'Persuasive': 10704,
 'Unconvincing': 300}

In [72]:
data['rate_rank'][0]

['Inspiring',
 'Funny',
 'Persuasive',
 'Fascinating',
 'Informative',
 'Ingenious',
 'Beautiful',
 'Jaw-dropping',
 'Courageous',
 'OK',
 'Longwinded',
 'Unconvincing',
 'Confusing',
 'Obnoxious']

In [81]:
data2[0]['transcripts'][:200]

"Good morning. How are you?(Laughter)It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.(Laughter)There have been three themes running through the conference whic"

In [97]:
nameList = data.columns

In [98]:
nameList

Index([u'comments', u'description', u'duration', u'event', u'film_date',
       u'languages', u'main_speaker', u'name', u'num_speaker',
       u'published_date', u'rate_dict', u'rate_percentage', u'rate_rank',
       u'ratings', u'related_talks', u'speaker_occupation', u'tags', u'title',
       u'transcripts', u'url', u'views'],
      dtype='object')